In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

In [ ]:
import time
import math

In [ ]:
%matplotlib inline 

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

# generate specific distribution

In [ ]:
num_samples = 10000
num_bins = 200

samples = np.random.random(size=num_samples)

_ = plt.hist(samples, num_bins)
plt.show()

In [ ]:
print(*dir(np.random))

**lognormal**
$$p(x) = \frac{1}{\sigma x \sqrt{2\pi}} e^{(-\frac{(ln(x)-\mu)^2}{2\sigma^2})}$$

$$f(x; a,b) = \frac{1}{B(\alpha, \beta)} x^{\alpha - 1} (1 - x)^{\beta - 1}$$

$$B(\alpha, \beta) = \int_0^1 t^{\alpha - 1}(1 - t)^{\beta - 1} dt$$

In [ ]:
samples = np.random.beta(a = 1, b = 2, size = 1000)
_ = plt.scatter(np.arange(len(samples)), samples)

In [ ]:
x = [{"index": 15, "kind": 1}, {"index": 11, "kind": 2}]
for i in x}

In [ ]:
any(np.random.gamma(shape = 1, scale = 0.1, size = 1000) < 0)

In [ ]:
samples = np.random.gamma(shape = 1, scale = 0.1, size = 1000) 
_ = plt.scatter(np.arange(len(samples)), samples)

In [ ]:
samples = np.random.uniform(low = -1, high = 1, size = 1000)
_ = plt.scatter(np.arange(len(samples)), samples)

In [ ]:
samples = np.random.standard_normal(1000)
_ = plt.plot(samples)

In [ ]:
samples = np.random.standard_normal(size = 1000)
_ = plt.scatter(np.arange(len(samples)), samples)

In [ ]:
np.random.poisson(lam = 2, size = 15)

In [ ]:
samples = np.random.poisson(lam = 2, size = 1000)
_ = plt.scatter(np.arange(len(samples)), samples)

In [ ]:
?np.random.normal

In [ ]:
distr = {
    "norm":
    lambda loc, scale, size: np.random.normal(
        loc=loc, scale=scale, size=size),
    "beta":
    lambda a, b, size: np.random.beta(a = a, b = b, size = size)
}
par = ("norm", 3, 1)
par[1:]
distr[par[0]](*par[1:], 10)

In [ ]:
class tem():
    def __init__(self):
        self._distr =  {"norm":lambda loc, scale, size: np.random.normal(loc=loc, scale=scale, size=size),
                        "beta":lambda a, b, size: np.random.beta(a = a, b = b, size = size)}
    def get_samples(self, par):
        print(self._distr[par[0]](*par[1:], size = 10))

x = tem()
x.get_samples(("norm", 3, 1))
x.get_samples(("beta", 2, 4))

In [ ]:
np.random.uniform(4, size = 2)*10
[1,2,3]*2

In [ ]:
mean = 3600
std = 1600
num_samples = 1000
samples = np.random.normal(loc = 0.5, scale = 0.1, size=num_samples)
_ = plt.scatter(np.arange(len(samples)), samples)

In [ ]:
np.random.random(size = 30000).mean()

In [ ]:
np.arange(10) +np.arange(10)

# `np.random`

In [ ]:
?np.random.randint

In [ ]:
x = np.random.randint(low = 1, high = 10, size = (3,3))
x
x[1,2]

In [ ]:
np.random.randint(low = 1, high = 10, size = 5)
np.random.random(size = (3, 3))
np.random.random_sample(size = (1,3))
np.random.choice(a = 60, size = 10)

In [ ]:
def corr2d(X, K):
    """Compute 2D cross-correlation."""
    h, w = K.shape
    Y = np.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i: i + h, j: j + w] * K).sum()
    return Y

X = np.random.randint(1, 10, [3, 3])
K = np.random.randint(1, 10, [2, 2])
corr2d(X, K)

X = np.random.randint(1, 10, [4, 4])
K = np.random.randint(1, 10, [2, 2])
corr2d(X, K)

In [ ]:
?np.random.randn

In [ ]:
x = list(map(np.random.randn, [10**i for i in range(1, 5)]))
[(i.mean(), i.var()) for i in x]

A new array holding the result is returned unless out is specified, in which case it is returned.

In [ ]:
?np.random.rand

In [ ]:
x = 2 * np.random.rand(10)
x
x.cumsum()

In [ ]:
nt = 10
t = 2 * np.random.rand(10 ).cumsum()
t.max() 
x = np.random.randn(nt)
x.max()
_ = plt.scatter(t, x)

In [ ]:
t
tnew

In [ ]:
# Make a uniform grid, with the same number of points:
tnew = np.linspace(t.min(), t.max(), nt)
xnew = np.interp(tnew, t, x)

_ = plt.scatter(tnew, xnew)
print("standard deviations: ", x.std(), xnew.std())

In [ ]:
import os
import pandas_datareader.data as web
import seaborn as sns
sns.set(style='ticks', context='talk')

if int(os.environ.get("MODERN_PANDAS_EPUB", 0)):
    import prep # noqa

In [ ]:
gs = web.DataReader("GS", data_source='yahoo', start='2006-01-01',
                    end='2010-01-01')
gs.head().round(2)

In [ ]:
type(gs)

In [ ]:
gs.resample("5d").mean().head()

# `np.zeros`

In [ ]:
np.argmin([1, 2, 0, 3, -2])

In [ ]:
?np.zeros

In [ ]:
matrix = np.zeros(shape = (3,4))

In [ ]:
np.inf >= np.inf + 1

In [ ]:
matrix[0,:] = np.inf 
matrix

In [ ]:
def dtw(series_1, series_2, norm_func = np.linalg.norm):
	matrix = np.zeros((len(series_1) + 1, len(series_2) + 1))
	matrix[0,:] = np.inf
	matrix[:,0] = np.inf
	matrix[0,0] = 0
	for i, vec1 in enumerate(series_1):
		for j, vec2 in enumerate(series_2):
			cost = norm_func(vec1 - vec2)
			matrix[i + 1, j + 1] = cost + min(matrix[i, j + 1], matrix[i + 1, j], matrix[i, j])
	matrix = matrix[1:,1:]
	i = matrix.shape[0] - 1
	j = matrix.shape[1] - 1
	matches = []
	mappings_series_1 = [list() for v in range(matrix.shape[0])]
	mappings_series_2 = [list() for v in range(matrix.shape[1])]
	while i > 0 or j > 0:
		matches.append((i, j))
		mappings_series_1[i].append(j)
		mappings_series_2[j].append(i)
		option_diag = matrix[i - 1, j - 1] if i > 0 and j > 0 else np.inf
		option_up = matrix[i - 1, j] if i > 0 else np.inf
		option_left = matrix[i, j - 1] if j > 0 else np.inf
		move = np.argmin([option_diag, option_up, option_left])
		if move == 0:
			i -= 1
			j -= 1
		elif move == 1:
			i -= 1
		else:
			j -= 1
	matches.append((0, 0))
	mappings_series_1[0].append(0)
	mappings_series_2[0].append(0)
	matches.reverse()
	for mp in mappings_series_1:
		mp.reverse()
	for mp in mappings_series_2:
		mp.reverse()
	
	return matches, matrix[-1, -1], mappings_series_1, mappings_series_2, matrix


In [ ]:
np.linalg.norm(x = [10,12])
dtw(series_1=[1,1,2],series_2=[1,1,2,3])

# `np.sum` 

In [ ]:
x = np.random.randint(1, 10, (2, 2))
x 
np.sum(x, axis = 1)
np.sum(x, axis = 0)

https://realpython.com/numpy-array-programming/

In [ ]:
array = np.random.randint(1, 10, (2, 4, 3))
array
for i in range(len(array.shape)):
    print("along axis_{} the sum is: \n {}".format(i, np.sum(array, axis=i)))
    print(" ---------------------------")

# built-in elementwise functions

In [ ]:
arr = np.random.randint(1, 10, 2)
arr
np.exp2(arr)
np.sqrt(arr)
np.log(arr)

https://stackoverflow.com/questions/57830433/how-to-get-the-euclid-distance-from-specified-array-in-numpy#57830445

In [ ]:
def find_eucledian_distances(a_points, b_points):
    return np.sqrt(np.sum((a_points - b_points)**2, axis=-1))


a = np.array([[3.0, 4.0], [-3.0, -6.0], [-2.5, 6.3]])
b = np.array([[0.0, 0.0], [2.0, 6.0], [4.5, -8.3]])
find_eucledian_distances(a, b)

<span style="font-family: New York Times; font-size: 1em; color: green;">
    
you should `np.sum` along the last axis. if you don't specific the axis=-1, the `np.sum()` will sum all of the elements.

# arithmetic between numpy array

https://stackoverflow.com/questions/57806304/numpy-elementwise-multiplication-with-multiple-columns#57806420

In [ ]:
a = np.array([[1, 2, 3], [4, 5, 6]])
b = np.array([[7, 8, 9, 10], [11, 12, 13, 14]])
c = np.zeros((2, 4, 3))

for i, row in enumerate(b):
    for j, col in enumerate(row):
        print('Mult:', a[i, :], '*', col)
        c[i, j, :] = a[i, :] * col

print(c[0, :, :])
print(c[1, :, :])

In [ ]:
a = np.array([[1, 2, 3], [4, 5, 6]])
b = np.array([[7, 8, 9, 10], [11, 12, 13, 14]])
print([item * k for item in a for k in b.ravel()])

In [ ]:
a = np.array([[1, 2, 3], [4, 5, 6]])
b = np.array([[7, 8, 9, 10], [11, 12, 13, 14]])
a[:,None,:] * b[:,:,None]

https://stackoverflow.com/questions/57843027/add-small-np-array-to-larger-np-array#57843046

In [ ]:
import numpy as np
a = np.arange(1, 11)
b = np.arange(1,3)
print('a:',a)
print('b:',b)
a[0::2] += b[0]
a[1::2] += b[1]
print('a:',a)

In [ ]:
a = np.arange(1, 11)
b = np.arange(1,3)
a.reshape(-1, len(b)) 
a.reshape(-1, len(b)) + b 
(a.reshape(-1, len(b)) + b).ravel()

# `np.reshape`

https://docs.scipy.org/doc/numpy/reference/generated/numpy.reshape.html#numpy.reshape

In [ ]:
a = np.array([[1,2,3, 4], [5,6,7,8]])
print(np.reshape(a, (-2)) )
print("___________________________________")
print(np.reshape(a, (2, 2,-2)) )
print("___________________________________")
print(np.reshape(a, (2, -1,4)) )
np.reshape(a,(1,-1))

# `np.array.tolist`

In [ ]:
arr = np.array([[1, 1, 1, 1], [2, 2, 2, 2], [3, 3, 3, 3], [4, 4, 4, 4]])
lens = [1, 2, 3, 4]
[a[:i].tolist() for a, i in zip(arr, lens)]

In [ ]:
num1 = [2, 2, 0]
num2 = [1, 0]
list(str( int(''.join(map(str, num1))) * int(''.join(map(str, num2)))))

In [ ]:
def multiply(num1, num2):
    if num1 == [0] or num2 == [0]:
        return [0]
    sign = -1 if (num1[0] < 0) ^ (num2[0] < 0) else 1
    num1[0] = abs(num1[0])
    num2[0] = abs(num2[0])
    res = [0] * (len(num1) + len(num2) + 1) # space O(n + m)
    for i in range(len(num1) - 1, -1, -1):
        for j in range(len(num2) - 1, -1, -1):
            res[i + j + 1] += num1[i] * num2[j]
            res[i + j] += res[i + j + 1] // 10
            res[i + j + 1] %= 10
    res[0] *= sign

    #res = res[next((i for i, x in enumerate(res) if x != 0), len(res)):] or [0]
    return res

multiply(num1, num2)

# `np.mean`

https://stackoverflow.com/questions/53178018/average-of-elements-in-a-subarray

https://stackoverflow.com/questions/57814566/get-average-of-the-numpy-ndarray

In [ ]:
m = np.array([[[0, 1, 2], [3, 4, 5], [6, 7, 8]],
              [[9, 10, 11], [12, 13, 14], [15, 16, 17]],
              [[18, 19, 20], [21, 22, 23], [24, 25, 26]]])

In [ ]:
m.shape
np.mean(m, axis=0)
np.mean(m, axis=(0,1))
np.mean(m, axis=2)
m[0]

In [ ]:
[np.mean(m[i]) for i in range(m.shape[0])]
list(map(np.mean, m))
np.mean(m, (1, 2))

In [ ]:
np.ones(shape = (2,1), order = "F")
np.ones(shape = (2,1), order = "C", dtype = int)

In [ ]:
?np.ones

# `np.where`

In [ ]:
shape = (3,3,3)
m = np.random.randint(1,10,(shape))

In [ ]:
np.where(m == np.amax(m)) 

In [ ]:
a = np.array([1,2,3,4])
x, = np.where(a==a.max())
print(x)

In [ ]:
data = [[0.4, 1.5, 2.6],
        [3.4, 0.2, 0.0],
        [np.nan, 3.2, 1.0],
        [1.0, 4.6, np.nan]]

In [ ]:
np.where(np.isnan(data))

In [ ]:
x = np.array(data)
np.isnan(x) 

In [ ]:
import warnings
warnings.filterwarnings('ignore')
data = [[0.4, 1.5, 2.6],
        [3.4, 0.2, 0.0],
        [np.nan, 3.2, 1.0],
        [1.0, 4.6, np.nan]]
x = np.array(data) 
i, j = np.where(x > 1 )
warnings.filterwarnings('ignore')
for a, b in zip(i, j):
    print('lon: {} lat: {} value: {}'.format(a, b, x[a,b]))

In [ ]:
array= np.array([[1, 2, 3],[np.nan, np.nan, np.nan], [3, 4, 5]])
np.nan_to_num(array)

In [ ]:
A = np.array([[ 0.57383254,  0.10132767,  0.86211639,  0.35402222],
       [ 0.20238346,  0.93204519,  0.84563318,  0.68373515],
       [ 0.5795946 ,  0.24307856,  0.56676058,  0.08502582],
       [ 0.27188428,  0.0630682 ,  0.9762359 ,  0.50456657],
       [ 0.6522969 ,  0.85018875,  0.22728716,  0.82851854]]) 
target = np.array([ 0.57959463 ,  0.24307856,  0.56676058,  0.08502582])

In [ ]:
np.where(A==target)[0]

In [ ]:
result, = np.where(np.all(A == target, axis=1))
print(result)

https://stackoverflow.com/questions/57833856/how-to-convert-ax-y-z-to-x0-xn-y0-yn-z0-zn

In [ ]:
A = np.ones((1,2))
np.where(A)[0]
np.where(A)[1]
np.ravel(A)

# `np.rotate`

In [ ]:
x = np.arange(25).reshape(5, -1) 
rotate(x, angle=45, reshape=False)

In [ ]:
def switch1(s):
    s = list(s)
    for i in range(len(s)):
        if s[i] == 'a':
            s[i] = 'b'
        if s[i] == 'b':
            s[i] = 'a'
    return s

def switch2(s):
    s = list(s)
    for i in range(len(s)):
        if s[i] == 'a':
            s[i] = 'b'
        elif s[i] == 'b':
            s[i] = 'a'
    return s

switch1('abab')
switch2('abab')

# `np.searchsorted`

* https://stackoverflow.com/questions/57121825/vectorized-approach-to-extract-the-rows-of-the-end-point-array-that-contains-the

In [ ]:
np.random.seed(0)

size = 32000
base_arr = np.arange(size)*10
base_arr
t1 = np.random.randint(0,6, size)+base_arr
t2 = np.random.randint(5,10, size)+base_arr

A = np.vstack((t1,t2)).T
V = np.sort(np.random.randint(0,10,3*size)+np.repeat(base_arr,3))

In [ ]:
A[np.searchsorted(v,A[:,0],'left')!=np.searchsorted(v,A[:,1],'right')]

In [ ]:
def find_bounding_intervals(A, v):
    rows = []
    i = 0
    for row in A:
        while all(v[i] < row):
            i += 1
        if row[0] <= v[i] <= row[1]:
            rows.append(row)
    return np.array(rows)
find_bounding_intervals(A, V)

# `np.consum`

In [ ]:
x = np.array([[np.nan, np.nan, np.nan, np.nan, np.nan],
       [np.nan, np.nan, np.nan, np.nan, np.nan],
       [ 0., np.nan, np.nan, np.nan, np.nan],
       [ 0., np.nan,  0., np.nan,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [np.nan,  0.,  0.,  0.,  0.]])
x.T
np.isnan(x)
np.nonzero(x)
np.where(np.isnan(x))

x[np.nonzero(x)] = 2 
x 

In [ ]:
x = np.array([[5.72756493, 0.        , 0.        ],
          [2.86378246, 4.96021673, 0.        ],
          [5.72756493, 3.30681115, 2.33826859]])

https://stackoverflow.com/questions/57966043/replace-zeros-in-an-array-with-a-continuous-sequence-of-integers


In [ ]:
arr = np.array([[np.nan, np.nan, np.nan, np.nan, np.nan],
       [np.nan, np.nan, np.nan, np.nan, np.nan],
       [ 0., np.nan, np.nan, np.nan, np.nan],
       [ 0., np.nan,  0., np.nan,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [np.nan,  0.,  0.,  0.,  0.]])

mask = ~np.isnan(arr)
np.nan_to_num(arr + 1).ravel('F')
np.nan_to_num(arr + 1).ravel('F').cumsum()
# Applying ravel function in F-style ordering
`b
arr[mask] = np.nan_to_num(arr + 1).ravel('F').cumsum().reshape(arr.shape, order='F')[mask]
arr 

# statistical value 

In [ ]:
x = np.arange(6)
print("\nOriginal array:")
print(x)
r1 = np.mean(x)
r2 = np.average(x)
assert np.allclose(r1, r2)
print("\nMean: ", r1)
r1 = np.std(x)
r2 = np.sqrt(np.mean((x - np.mean(x)) ** 2 ))
assert np.allclose(r1, r2)
print("\nstd: ", 1)
r1= np.var(x)
r2 = np.mean((x - np.mean(x)) ** 2 )
assert np.allclose(r1, r2)
print("\nvariance: ", r1)